In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [16]:
class Vertice:
    
    def __init__(self, valor, direcionado=True):
        self.__valor = valor
        self.__direcionado = direcionado
        self.__arestas = set()
    
    def getValor(self):
        return self.__valor
    
    def setValor(self, valor):
        self.__valor = valor
        
    def getArestas(self):
        return self.__arestas
    
    def adicionarAresta(self, aresta):
        self.__arestas.add(aresta)
        
    def getArestasSaida(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasDeSaida = []
        for aresta in self.__arestas:
            if aresta.getvOrigem() == self:
                arestasDeSaida.append(aresta)
        return arestasDeSaida
    
    def getArestasEntrada(self):
        if self.__direcionado == False:
            return self.__arestas
        arestasSaida = []
        for aresta in self.__arestas:
            if aresta.getvDestino() == self:
                arestasSaida.append(aresta)
        return arestasSaida
    
    def getGrau(self):
        return len(self.getArestasSaida())+ len(self.getArestasEntrada())
    
    def getAdjacentes(self, v):
        listaVerticesAdjacentes = []
        for arestas_de_saida in v.getArestasSaida():
            listaVerticesAdjacentes.append(arestas_de_saida.getvDestino())
        return listaVerticesAdjacentes

In [17]:
class Aresta:
    def __init__(self, vOrigem, vDestino, peso, direcionada=True):
        self.__vOrigem = vOrigem
        self.__vDestino = vDestino
        self.__peso = peso
        self.__direcionada = direcionada
        self.__vOrigem.adicionarAresta(self)
        self.__vDestino.adicionarAresta(self)
        
    def getvOrigem(self):
        return self.__vOrigem
    def getvDestino(self):
        return self.__vDestino
    def getValor(self):
        return self.__peso

In [18]:
from collections import deque
class Grafo:
    def __init__(self, direcionado=True):
        self.__vertices = set()
        self.__arestas  = set()
        self.__direcionado = direcionado
        
    def setVertices(self, vertices):
        self.__vertices = vertices
        
    def setArestas(self, arestas):
        self.__arestas = arestas
        
    def getVertices(self):
        return self.__vertices
    
    def getVerticeByValor(self, valor):
        for v in self.__vertices:
            if v.getValor() == valor:
                return v
        return None
    
    def getArestas(self):
        return self.__arestas
    
    def checkHandShakingLemma(self):
        somaGraus = 0
        for v in self.getVertices():
            somaGraus+= v.getGrau()
        if somaGraus == len(self.getArestas())*2:
            return True
        else:
            return False
        
    def dfs(self, graph, v, visitados=[]):
        if v not in visitados:
            visitados.append(v) 
        if len(v.getAdjacentes(v)) == 0:
            self.dfs(graph, next(iter(graph.getVertices())), visitados)
        else: # vertice escolhido tem adjacentes
            for adjacente in v.getAdjacentes(v): 
                if adjacente not in visitados: 
                    self.dfs(graph, adjacente, visitados) 
        return visitados
    
    def bfs(self, v, visitados = [], fila = deque([])):
        fila.append(v) 
        if v not in visitados: 
            visitados.append(v)
        while fila:  
            vertice = fila.popleft() 
            if len(vertice.getArestasSaida()) == 0: 
                self.bfs(next(iter(self.getVertices())), visitados, fila)    
            else:
                for e in vertice.getArestasSaida(): 
                    vAdjacente = e.getvDestino()
                    if vAdjacente not in visitados: 
                        visitados.append(vAdjacente)  
                        fila.append(vAdjacente)  
        return visitados
    
    def buscarPorValor(self, valor):
        for v in self.bfs(next(iter(self.getVertices())), visitados = [], fila = deque([])):
            if valor == v.getValor():
                return valor
        return None
    
    def Euler(self, graph):
        impar = 0
        for v in graph.getVertices():
            if v.getGrau() %2 != 0:
                impar += 1
        if impar != 0 >= 2:
            return True 
        else:
            return False

      

In [19]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True )

In [20]:
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5})
G.setArestas({a1, a2, a3, a4, a5})

In [21]:
for v in G.getVertices():
    print(v.getValor(), end="\t")

5	3	4	1	2	

In [22]:
for a in G.getArestas():
    print(a.getvOrigem().getValor(), end="")
    print(" ---> ", end="")
    print(a.getvDestino().getValor(), end="\t")

1 ---> 2	4 ---> 5	2 ---> 3	4 ---> 1	3 ---> 4	

In [23]:
v1.getGrau()

2

In [24]:
G.checkHandShakingLemma()

True

In [25]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
for vertice in G.getVertices():
    print(f"Busca em profundidade, iniciando com o vértice {vertice.getValor()}:")
    for v in G.dfs(G, vertice, visitados=[]):
        print(str(v.getValor())+"\t", end="")
    print("\n.................................................") 

Busca em profundidade, iniciando com o vértice 2:
2	3	4	5	6	1	
.................................................
Busca em profundidade, iniciando com o vértice 5:
5	2	3	4	6	1	
.................................................
Busca em profundidade, iniciando com o vértice 3:
3	4	5	2	6	1	
.................................................
Busca em profundidade, iniciando com o vértice 6:
6	2	3	4	5	1	
.................................................
Busca em profundidade, iniciando com o vértice 4:
4	5	2	3	6	1	
.................................................
Busca em profundidade, iniciando com o vértice 1:
1	2	3	4	5	6	
.................................................


In [26]:
v1 = Vertice(1)
v2 = Vertice(2)
v3 = Vertice(3)
v4 = Vertice(4)
v5 = Vertice(5)
v6 = Vertice(6)
a1 = Aresta( v1, v2, 10, True )
a2 = Aresta( v2, v3, 20, True )
a3 = Aresta( v3, v4, 30, True )
a4 = Aresta( v4, v1, 40, True )
a5 = Aresta( v4, v5, 50, True ) 
a6 = Aresta( v4, v6, 60, True ) 
G = Grafo()
G.setVertices({v1, v2, v3, v4, v5,v6})
G.setArestas({a1, a2, a3, a4, a5, a6})
for vertice in G.getVertices():
    print(f"Busca em largura, iniciando com o vértice {vertice.getValor()}:")
    for v in  G.bfs(vertice, visitados = [], fila = deque([])):
        print(str(v.getValor())+"\t", end="")
    print("\n............................................") 

Busca em largura, iniciando com o vértice 1:
1	2	3	4	5	6	
............................................
Busca em largura, iniciando com o vértice 4:
4	5	6	1	2	3	
............................................
Busca em largura, iniciando com o vértice 3:
3	4	5	6	1	2	
............................................
Busca em largura, iniciando com o vértice 2:
2	3	4	5	6	1	
............................................
Busca em largura, iniciando com o vértice 5:
5	1	2	3	4	6	
............................................
Busca em largura, iniciando com o vértice 6:
6	1	2	3	4	5	
............................................


In [27]:
if G.Euler(G):
    print("é um grafo euleriano")
else:
    print("não é um grafo euleriano")

não é um grafo euleriano


In [28]:
valor = int(input("Valor procurado: "))
if G.buscarPorValor(valor) == None:
   print(f"{valor} não encontrado no grafo")
else:
   print(f"{valor} encontrado no grafo")

Valor procurado: 7
7 não encontrado no grafo


In [29]:
import pathlib
pathlib.